In [14]:
import glob
import pandas as pd
import re

In [15]:
codon_to_aa = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
    'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W'
}

In [16]:
pattern = '../results/*Delta*/build_site_fubar_table/surface_glycoprotein.fubar.tsv'
df_list = []
for path in sorted(glob.glob(pattern)):
    parts = re.split(r'/|_', path)
    df = pd.read_csv(path, sep='\t')
    df = df.drop_duplicates(subset='Position_in_reference', keep='first').reset_index(drop=True)
    seq_columns = [col for col in df.columns if col.startswith('Seq_')]
    seq_df = df[seq_columns]
    row_value_counts = seq_df.apply(lambda row: row.value_counts().to_dict(), axis=1)
    df[f'{parts[2]}_{parts[3]}_codon_counts'] = row_value_counts
    df.rename(columns={'Prob[alpha<beta]': f'{parts[2]}_{parts[3]}_Prob[alpha<beta]'}, inplace=True)
    df['Ref_amino_acid'] = df['Ref_codons'].apply(lambda codon: codon_to_aa.get(codon, 'X'))
    df = df[['Position_in_reference', 'Ref_amino_acid', f'{parts[2]}_{parts[3]}_Prob[alpha<beta]', f'{parts[2]}_{parts[3]}_codon_counts']]
    df_list.append(df)
merged_df = df_list[0]
for df in df_list[1:]:
    merged_df = merged_df.merge(df, on=['Position_in_reference', 'Ref_amino_acid'])
merged_df

,Position_in_reference,Ref_amino_acid,2021-08_unvaccinated_Prob[alpha<beta],2021-08_unvaccinated_codon_counts,2021-08_vaccinated_Prob[alpha<beta],2021-08_vaccinated_codon_counts,2021-09_unvaccinated_Prob[alpha<beta],2021-09_unvaccinated_codon_counts,2021-09_vaccinated_Prob[alpha<beta],2021-09_vaccinated_codon_counts,...,2021-11_vaccinated_Prob[alpha<beta],2021-11_vaccinated_codon_counts,2021-12_unvaccinated_Prob[alpha<beta],2021-12_unvaccinated_codon_counts,2021-12_vaccinated_Prob[alpha<beta],2021-12_vaccinated_codon_counts,2022-01_unvaccinated_Prob[alpha<beta],2022-01_unvaccinated_codon_counts,2022-01_vaccinated_Prob[alpha<beta],2022-01_vaccinated_codon_counts
0,1,M,0.217039,{'ATG': 1724},0.314287,{'ATG': 748},0.150510,{'ATG': 2768},0.204489,{'ATG': 1478},...,0.152529,{'ATG': 2626},0.148683,{'ATG': 2670},0.153177,{'ATG': 2648},0.307831,{'ATG': 439},0.326073,{'ATG': 383}
1,2,F,0.520111,"{'TTT': 1723, 'TTA': 1}",0.355881,{'TTT': 748},0.484714,"{'TTT': 2767, 'TTA': 1}",0.258370,{'TTT': 1478},...,0.226105,{'TTT': 2626},0.478373,"{'TTT': 2669, 'TTA': 1}",0.217029,{'TTT': 2648},0.345322,{'TTT': 439},0.360582,{'TTT': 383}
2,3,V,0.284923,{'GTT': 1724},0.606077,"{'GTT': 747, 'GGT': 1}",0.258219,"{'GTT': 2765, 'GCT': 2, 'GCC': 1}",0.536248,"{'GTT': 1477, 'ATT': 1}",...,0.239830,{'GTT': 2626},0.658447,"{'GTT': 2668, 'GCT': 2}",0.648344,"{'GTT': 2646, 'GGT': 1, 'ATT': 1}",0.357258,{'GTT': 439},0.615549,"{'GTT': 382, 'GCT': 1}"
3,4,F,0.271632,{'TTT': 1724},0.355881,{'TTT': 748},0.215111,{'TTT': 2768},0.258370,{'TTT': 1478},...,0.226105,{'TTT': 2626},0.210352,{'TTT': 2670},0.217029,{'TTT': 2648},0.345322,{'TTT': 439},0.360582,{'TTT': 383}
4,5,L,0.998387,"{'CTT': 1675, 'TTT': 49}",0.910865,"{'CTT': 734, 'TTT': 14}",0.971938,"{'CTT': 2715, 'TTT': 50, 'CTC': 2, 'ATT': 1}",0.997586,"{'CTT': 1450, 'TTT': 27, 'ATT': 1}",...,0.997603,"{'CTT': 2569, 'TTT': 57}",0.999566,"{'CTT': 2616, 'TTT': 54}",0.996728,"{'CTT': 2600, 'TTT': 48}",0.992207,"{'CTT': 425, 'TTT': 14}",0.919761,"{'CTT': 378, 'TTT': 5}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,1269,K,0.295548,{'AAA': 1724},0.369542,{'AAA': 748},0.241996,{'AAA': 2768},0.280843,{'AAA': 1478},...,0.245654,{'AAA': 2626},0.241256,{'AAA': 2670},0.240216,{'AAA': 2648},0.356263,{'AAA': 439},0.368297,{'AAA': 383}
1269,1270,L,0.317908,{'TTA': 1724},0.385899,{'TTA': 748},0.273395,{'TTA': 2768},0.303403,{'TTA': 1478},...,0.286294,{'TTA': 2626},0.268103,{'TTA': 2670},0.038417,"{'TTA': 2646, 'CTA': 2}",0.120776,"{'TTA': 438, 'CTA': 1}",0.383521,{'TTA': 383}
1270,1271,H,0.286330,{'CAT': 1724},0.364520,{'CAT': 748},0.230054,{'CAT': 2768},0.271921,{'CAT': 1478},...,0.237619,{'CAT': 2626},0.080920,"{'CAT': 2669, 'CAC': 1}",0.229335,{'CAT': 2648},0.353678,{'CAT': 439},0.367400,{'CAT': 383}
1271,1272,Y,0.065931,"{'TAC': 1722, 'TAT': 2}",0.404773,{'TAC': 748},0.132770,"{'TAC': 2767, 'TAT': 1}",0.322692,{'TAC': 1478},...,0.062277,"{'TAC': 2623, 'TAT': 3}",0.306176,{'TAC': 2670},0.314437,{'TAC': 2648},0.394980,{'TAC': 439},0.404975,{'TAC': 383}


In [20]:
prob_pairs = {}
for col in merged_df.columns:
    if '[alpha<beta]' in col:
        key = col.split('_')[0]
        prob_pairs.setdefault(key, []).append(col)

diff_df = pd.DataFrame()
for key, value in prob_pairs.items():
    if len(value) == 2:
        diff_df[f'{key}_diff'] = (merged_df[value[1]] - merged_df[value[0]])

merged_df['[alpha<beta]_max_diff'] = diff_df.max(axis=1)
merged_df['[alpha<beta]_max_diff'] = diff_df.apply(lambda row: max(row, key=abs), axis=1)

merged_df['[alpha<beta]_mean_diff'] = diff_df.mean(axis=1)

In [18]:
merged_df.to_csv('../spike_fubar_delta.tsv', sep='\t', index=False)